<a href="https://colab.research.google.com/github/anahimaciel/Networks-in-cities/blob/main/RadiusCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geopy.distance

In [111]:
city= "Taubaté"

# Prep

In [112]:
#read df
df = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/filtered_cdr.txt",usecols=[1,2,3,4,5])
df

,date,time,user_from,user_to,antenna
0,2013-04-16,17:49:07,B210CA503735D00F84FF481FF5A590E5,3B2DBB756C7200E5ED7CAB8E06B2441F,56219
1,2013-04-16,16:17:16,B210CA503735D00F84FF481FF5A590E5,A49BA005920B8BA24BEE40F455B69B9B,56219
2,2013-04-16,11:58:00,0ECCF8C78516468DC4C725E0C8DCE630,F8BFFB2E7CA8B8058AD900E6AC7BB655,32207
3,2013-04-16,08:41:55,82418C4C787941B4B3E0189B84BC44F9,4B4C15729C19DFC43C1C5FEA4D7CF409,56268
4,2013-04-16,11:17:19,07A785880CA135F0F8BA511464AEB65F,CB3E1C253D20D4FA65EDC2334778320E,56198
...,...,...,...,...,...
426976,2013-04-03,18:24:37,748A26E82EA691AFF4F9A868EC069ACE,1212AA53598E0EBA69CFA3AFC09EE491,56189
426977,2013-04-03,20:53:48,5EFC7B2DA91D735F4C826E8C9CE0D683,AEA3556E1EF6CC568646CCEBF1750DAF,56259
426978,2013-04-03,08:43:57,562E477CC86201CE17BADB0A0C2CA39C,08C187611706D2513FE8A277D8492EEF,56189
426979,2013-04-03,08:10:58,562E477CC86201CE17BADB0A0C2CA39C,3D2454AA32276690E0E663AE36F833A9,56189


In [113]:
#read antennas files
map = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/antennas_"+city+".txt", sep=';')
map

,CELLID,UF,DDD,LAT,LONG,CITY,MUNICIPALITY
0,14869,SP,12,-23.04436,-45.62864,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
1,29507,SP,12,-23.02480,-45.58230,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
2,60437,SP,12,-23.04879,-45.59704,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
3,21488,SP,12,-23.00489,-45.55462,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
4,34338,SP,12,-23.02322,-45.56945,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
...,...,...,...,...,...,...,...
123,56168,SP,12,-23.04417,-45.57844,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
124,56208,SP,12,-23.05421,-45.56397,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
125,56158,SP,12,-23.02579,-45.54841,Taubaté,Região Imediata de Taubaté-Pindamonhangaba
126,56178,SP,12,-23.03542,-45.56999,Taubaté,Região Imediata de Taubaté-Pindamonhangaba


# Compute home locations

In [114]:
#convert to datetime object
df['date']=pd.to_datetime(df['date'])
#create column with days of the week
df['day_week']=df['date'].dt.day_name()

#create dataframe with calls in weekends and nights
night_df=df[(df['day_week']=='Saturday') | (df['day_week']=='Sunday') | (df['time']> '19:00:00') | (df['time']< '06:00:00')]
night_df

,date,time,user_from,user_to,antenna,day_week
6,2013-04-16,21:11:30,63EF39AC5919943E8D8621E043A9CB8C,374FF51795FE8DD1A9500FFBF4732293,56317,Tuesday
16,2013-04-16,20:47:59,63EF39AC5919943E8D8621E043A9CB8C,8464D286412CC2FC3741171FF5A72A83,56317,Tuesday
19,2013-04-16,21:20:34,190A03E8B2326D51189AEA765722A2DD,A49BA005920B8BA24BEE40F455B69B9B,56289,Tuesday
26,2013-04-16,21:53:44,23C9532BCB3A25C7F33D056796D9B1A5,A62A538AFB8F16965B95AE6B575F040A,56198,Tuesday
30,2013-04-16,22:28:10,04F614E32888DB1D3FA37681B4E439F8,381E77DCA7DEFCC9450970FAF130BE9B,32197,Tuesday
...,...,...,...,...,...,...
426941,2013-04-03,19:30:31,748A26E82EA691AFF4F9A868EC069ACE,7D74C1AB3006A50FE9C0028A449B7FFA,56189,Wednesday
426952,2013-04-03,21:33:18,F311382CFD821AD4EF8C6EDAE87AB513,E4D67990722B4CD280F390B7C1E4A45B,34337,Wednesday
426964,2013-04-03,20:46:42,3DEE6A065B0F921D71CAFFDFD49DEBDA,924DC7DC9546A3A03478FF6E29BE044B,56189,Wednesday
426977,2013-04-03,20:53:48,5EFC7B2DA91D735F4C826E8C9CE0D683,AEA3556E1EF6CC568646CCEBF1750DAF,56259,Wednesday


In [115]:
#create dataframe with data necessary to compute places of residence
residence_df=night_df.groupby(['user_from','antenna']).size().reset_index(name='time')
residence_df

,user_from,antenna,time
0,00029290B2E338EDAD26C95D60D28207,56298,1
1,0003632FCD126989D9A4650F465FC5B0,56257,1
2,0004B07E52EE2CFEF7B5F1A3F0FACE5C,56177,4
3,0004B07E52EE2CFEF7B5F1A3F0FACE5C,56198,1
4,0004B07E52EE2CFEF7B5F1A3F0FACE5C,56199,2
...,...,...,...
39689,FFFBD0B10A46EDB2E655CBADE50D9813,14929,1
39690,FFFD4EC7C7AF62041C15D3E608670EDB,56259,1
39691,FFFD4EC7C7AF62041C15D3E608670EDB,56317,7
39692,FFFD4EC7C7AF62041C15D3E608670EDB,56318,2


In [116]:
#compute places of residence
#drop locations other than the first
residence_df=residence_df.sort_values(['user_from','time'],ascending=[True,False]).drop_duplicates('user_from')
#only keep users whose first location complies with certain parameters (more than half of the calls of the user)
number_calls_map=residence_df['user_from'].map(df.groupby('user_from').size())
filter=(residence_df['time']>number_calls_map/2)
residence_df=residence_df[filter]

#include geographical location of antennas in residence dataframe
map=map.rename(columns={"CELLID": "antenna"})
residence_df=pd.merge(residence_df,map,on="antenna",how="left")

#delete unnecessary columns
residence_df=residence_df.drop(['time','UF','DDD','CITY','MUNICIPALITY'],axis=1)

residence_df.sort_values('antenna')

,user_from,antenna,LAT,LONG
5309,810F66EC40EBDFE30A51DBEF744814E6,14837,-23.04529,-45.61094
4296,673EF1340F51189DCBF2EA88DF04D343,14837,-23.04529,-45.61094
7956,C08CA4C9D8F115C65F02A122E15D92B9,14837,-23.04529,-45.61094
439,0A087C6F3D49775DD91E0C80F4228945,14837,-23.04529,-45.61094
9741,EB80A7B16B9412171D27E2FD57EF7489,14837,-23.04529,-45.61094
...,...,...,...,...
3337,4F08EA04440F04AE8215186B75D2C8ED,60469,-23.02378,-45.58518
5828,8E1C0C5048AC101D838651238D64F30F,60469,-23.02378,-45.58518
1086,18D9F38762DE763F1026D6A1EFD16128,60469,-23.02378,-45.58518
98,01F54C649E143EC4F130ECD9E9D40F01,60469,-23.02378,-45.58518


In [117]:
#extract from df users whose place of residence were not found (useful for next step)
filter=df['user_from'].isin(residence_df['user_from']) & df['user_to'].isin(residence_df['user_from'])
filtered_df=df[filter]
filtered_df

,date,time,user_from,user_to,antenna,day_week
317,2013-04-16,08:41:13,5D6A161BB218D27D4818B27101A1B743,472A7C65F2386EE48F48718039E965C6,56298,Tuesday
1417,2013-04-16,16:16:45,72E800D664B056285B9EB3FE75CB5855,BCB996836069538BFB7647A9B8C8A545,56299,Tuesday
1435,2013-04-16,18:58:17,BE80D436872F92C49034D3B2C6EE4D3C,23F268FE59A44321B3BCB0098F64D522,56177,Tuesday
2251,2013-04-16,08:09:17,CE0CB56F10F9730FE83F47A2103DCAD5,955BE272662AACCB7BA89EF4B7F7F59E,32207,Tuesday
2253,2013-04-16,17:42:24,CE0CB56F10F9730FE83F47A2103DCAD5,955BE272662AACCB7BA89EF4B7F7F59E,32207,Tuesday
...,...,...,...,...,...,...
420533,2013-04-03,20:28:51,0378103F420AAB74E22FD9912647DA7F,B54BC035289C025C23E792960FE9955B,29507,Wednesday
420915,2013-04-03,20:57:16,0378103F420AAB74E22FD9912647DA7F,B54BC035289C025C23E792960FE9955B,29508,Wednesday
423569,2013-04-03,19:15:17,120CE3BB1A4E27AC10665838489CA425,4BEEBF765D8702C92203342C7E4D60A2,60458,Wednesday
423873,2013-04-03,18:51:06,1DB414DEBBE7C10DDADE54A598DD88FC,2F32A2FF996C12154ADA67C6FE5299E2,56249,Wednesday


# Calculate average radius

In [118]:
#dicts with coordinates
dict_lat = residence_df.set_index('user_from')['LAT'].to_dict()
dict_long = residence_df.set_index('user_from')['LONG'].to_dict()

#function to calculate distance between residences of caller and receiver for each call
def get_distance(row):
    coords_caller=(dict_lat[row.user_from],dict_long[row.user_from])                #deixar mais rapido: agrupar por pessoa q ta ligando (n sei se ajudaria mt)
    coords_receiver=(dict_lat[row.user_to],dict_long[row.user_to])
    return geopy.distance.distance(coords_caller,coords_receiver).km

#apply function to filtered_df
filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)
filtered_df

<ipython-input-118-bd0efc1160f5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)


,date,time,user_from,user_to,antenna,day_week,distance
317,2013-04-16,08:41:13,5D6A161BB218D27D4818B27101A1B743,472A7C65F2386EE48F48718039E965C6,56298,Tuesday,9.169516
1417,2013-04-16,16:16:45,72E800D664B056285B9EB3FE75CB5855,BCB996836069538BFB7647A9B8C8A545,56299,Tuesday,2.447042
1435,2013-04-16,18:58:17,BE80D436872F92C49034D3B2C6EE4D3C,23F268FE59A44321B3BCB0098F64D522,56177,Tuesday,2.783347
2251,2013-04-16,08:09:17,CE0CB56F10F9730FE83F47A2103DCAD5,955BE272662AACCB7BA89EF4B7F7F59E,32207,Tuesday,0.000000
2253,2013-04-16,17:42:24,CE0CB56F10F9730FE83F47A2103DCAD5,955BE272662AACCB7BA89EF4B7F7F59E,32207,Tuesday,0.000000
...,...,...,...,...,...,...,...
420533,2013-04-03,20:28:51,0378103F420AAB74E22FD9912647DA7F,B54BC035289C025C23E792960FE9955B,29507,Wednesday,0.000000
420915,2013-04-03,20:57:16,0378103F420AAB74E22FD9912647DA7F,B54BC035289C025C23E792960FE9955B,29508,Wednesday,0.000000
423569,2013-04-03,19:15:17,120CE3BB1A4E27AC10665838489CA425,4BEEBF765D8702C92203342C7E4D60A2,60458,Wednesday,3.613466
423873,2013-04-03,18:51:06,1DB414DEBBE7C10DDADE54A598DD88FC,2F32A2FF996C12154ADA67C6FE5299E2,56249,Wednesday,6.058976


In [119]:
#drop calls to the same receiver (nao sei pq)
groups=filtered_df.groupby(['user_from','user_to'])
aux=groups.first()
aux=aux.groupby('user_from')['distance'].sum().to_frame(name='total_distance_without_alpha')

#calculate average radius for each user
radii=filtered_df.groupby("user_from").agg({'distance':['sum','size'],'user_to':'nunique'})
radii.columns=['total_distance','n_calls','contacts']
radii['avg_dist_by_calls']=radii['total_distance']/radii['n_calls']
radii['avg_radius']=radii['total_distance']/radii['contacts']
radii['total_distance_without_alpha']=aux['total_distance_without_alpha']
radii['avg_radius_without_alpha']=aux['total_distance_without_alpha']/radii['contacts']
radii

,total_distance,n_calls,contacts,avg_dist_by_calls,avg_radius,total_distance_without_alpha,avg_radius_without_alpha
user_from,,,,,,,
000DCF37F707122E7880307429D0A661,0.000000,1,1,0.000000,0.000000,0.000000,0.000000
0077CF6772C97F14AC51EECFAFE5561D,6.092165,1,1,6.092165,6.092165,6.092165,6.092165
00942CDF0A3CA173954CA9663B34FC36,0.000000,1,1,0.000000,0.000000,0.000000,0.000000
00E2803468F06B0FDDABD2D86D20FC69,0.000000,4,1,0.000000,0.000000,0.000000,0.000000
01089984DCB5574F784A75845C2CFE6A,1.352215,1,1,1.352215,1.352215,1.352215,1.352215
...,...,...,...,...,...,...,...
FE49F4EE0062F87A19E1C9CC71081F2A,5.826394,1,1,5.826394,5.826394,5.826394,5.826394
FE675206A57B6A234B27E03A6ECF4941,0.000000,1,1,0.000000,0.000000,0.000000,0.000000
FE7719E8F8815758FADF3BDBCD846FC7,6.522445,1,1,6.522445,6.522445,6.522445,6.522445


In [120]:
#results
with open("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/results.txt",'a') as file:
  file.truncate(0)
  for i in radii.columns:
    results_df=radii.agg({i:['max','min','mean']})
    file.write(results_df.to_string(index=False)+"\n")
    print(results_df)

      total_distance
max       515.555685
min         0.000000
mean        4.769301
         n_calls
max   254.000000
min     1.000000
mean    2.231767
      contacts
max   4.000000
min   1.000000
mean  1.055105
      avg_dist_by_calls
max            13.58666
min             0.00000
mean            2.09086
      avg_radius
max   515.555685
min     0.000000
mean    4.025003
      total_distance_without_alpha
max                      28.533425
min                       0.000000
mean                      2.238199
      avg_radius_without_alpha
max                  13.586660
min                   0.000000
mean                  2.100154


In [121]:
#saving demography data
with open("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/demo2.txt",'a') as file:
  file.truncate(0)
  file.write("users with residence: "+str(len(residence_df))+"\n")

In [122]:
#saving results
residence_df.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/home_locations.txt")
filtered_df.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/filtered_cdr_2.txt")
radii.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/radii.txt")

#for the networks
edges=groups.size().reset_index(name='weight')
edges.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/edges.txt")